In [1]:
import pandas as pd

from pathlib import Path
repo_path = Path('/home/krajda/anticipatio/')

In [2]:
tweets = pd.read_pickle(repo_path / 'data/final.pkl')
docs = tweets['txt'].tolist()
docs

[' Zero.',
 ' Any that only needs one world.',
 'The many-worlds interpretation is the most egregious violation of Occam’s razor in history.',
 'The fight over what AIs say and do has just started, and will never end.',
 "A must-read: 's thoughts on a life in probabilistic AI.\n",
 'For better or worse, social media is where society decides what to pay attention to.',
 "If everyone who thinks they're the only anti-woke in their organization got together, they'd be the majority.",
 'What Google is singularly lacking is someone who knows how to play the PR game, like Sam Altman or Demis Hassabis (or Elon Musk or Steve Jobs).',
 ' Wokeism is least popular among Gen Z of all age cohorts, according to surveys.',
 ' Across the board.',
 ' BERT.',
 ' On the contrary. No one hates wokeism more than Gen Z.',
 ' Applying n algorithms to m datasets.',
 ' I mean BERT.',
 'Many people don’t seem to know that Google has used a large language model in its search engine for years.',
 'The ratio of new

In [3]:
doc_topic = pd.read_csv(repo_path / 'data/topics_labels.csv')
doc_topic.sort_values('doc_id',inplace=True)
tweets['Topic'] = doc_topic['topic'].tolist()


In [4]:
emotions = pd.read_pickle(repo_path / 'data/emotions.pkl')

In [5]:
tweets = tweets.reset_index()
tweets['id'] = tweets.index
emotions['id'] = emotions.index

tweets = tweets.join(emotions.set_index('id'), on='id')

In [6]:
agg = tweets.groupby('Topic').mean().drop(columns=['id', 'index'])

/tmp/ipykernel_661312/2088533629.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  agg = tweets.groupby('Topic').mean().drop(columns=['id', 'index'])


In [7]:
print(agg.min(), agg.mean(), agg.max())

sadness         0.018535
disgust         0.029608
pessimism       0.016016
anger           0.026550
anticipation    0.144403
joy             0.093418
fear            0.012424
optimism        0.125394
surprise        0.034509
trust           0.028769
love            0.008637
dtype: float64 sadness         0.088257
disgust         0.135387
pessimism       0.053083
anger           0.122490
anticipation    0.460070
joy             0.496444
fear            0.086696
optimism        0.483553
surprise        0.085870
trust           0.128826
love            0.073711
dtype: float64 sadness         0.226947
disgust         0.630632
pessimism       0.145550
anger           0.643069
anticipation    0.687817
joy             0.915357
fear            0.481284
optimism        0.828290
surprise        0.158500
trust           0.254552
love            0.515652
dtype: float64


In [8]:
agg.to_csv(repo_path / 'data/to_paper/emotions_per_topic.csv')

In [9]:
anticipated_representatives = []

examples = 20

for n in range(100):
    x = tweets[tweets['Topic'] == n].nlargest(examples, ['anticipation'])[['anticipation', 'txt']]
    
    a = x['anticipation'].tolist()
    t = x['txt'].tolist()

    for i in range(examples):
        anticipated_representatives.append({'topic': n, 'anticipation': a[i], 'text': t[i]})
    
    
pd.DataFrame(anticipated_representatives).to_csv(repo_path / 'data/to_paper/anticipated_representatives.csv')

In [10]:
anticipated_representatives

[{'topic': 0,
  'anticipation': 0.9002268314361572,
  'text': "Will you join me on 4/28 for the next Data Masters Rendezvous: Shaping a Climate-Neutral Future with Data?  I'm curious to learn more from  the Open Footprint forum and  on the topic of #Sustainability  "},
 {'topic': 0,
  'anticipation': 0.897770345211029,
  'text': 'We want to hear from you!\n\nJoin our series of virtual and in-person meetings seeking to obtain public comments on the contents of the draft environmental impact statement for the Mars Sample Return Campaign being planned by  and  how:  '},
 {'topic': 0,
  'anticipation': 0.8881608247756958,
  'text': "Headlines from 2030s: Weather System Engineering firms gets green light from international bodies to develop tech that breaks up hurricanes. // Can't wait to see US Culture War version of this debate ;-) "},
 {'topic': 0,
  'anticipation': 0.8842005729675293,
  'text': "I'm interviewing world-renowned architect (and author of Cradle to Cradle) Bill McDonough fo

In [11]:
tweets['x'] = 1
topic_user = tweets[['Topic', 'user', 'x']].groupby(['Topic', 'user'], as_index=False)['x'].count().sort_values(by=['Topic','x'], ascending=False)

In [23]:
users_sum = topic_user.groupby('user')['x'].sum().reset_index()
users_sum.sort_values(by='x', ascending=False, inplace=True)
users_sum

,user,x
23,@EvanKirstel,77428
128,@digital_trans4m,37445
209,@psb_dc,36311
46,@KirkDBorne,33840
153,@jamesvgingerich,31682
...,...,...
40,@JesusSanSegundo,1
147,@ingliguori,1
361,localrat,1
6,@BigDataBRA,1


In [24]:
topic_user['share'] = topic_user.apply(lambda x: x['x'] / users_sum[users_sum['user'] == x['user']]['x'].values[0], axis=1)

In [25]:
topic_user['user_sum'] = topic_user.apply(lambda x: users_sum[users_sum['user'] == x['user']]['x'].values[0], axis=1)

In [30]:
topic_user2 = topic_user[topic_user['user_sum'] > 10]

tu2 = topic_user2.groupby('user')['share'].max().reset_index().sort_values(by='share', ascending=False)

In [45]:
topic_user2[topic_user['user'].isin(tu2[tu2['share'] > 0.5]['user'].tolist())].groupby('user').max().sort_values(by='share', ascending=False).to_csv(repo_path / 'data/to_paper/focused_users.csv')

In [35]:
tu2[tu2['share'] > 0.5]['user'].tolist()

['@peterfgallagher',
 'alphanmanas',
 'futurecheck',
 'Blazing_Science',
 'futuristufuk',
 '@timgrieveson',
 'elinafuturist',
 'meds888',
 'jeremygutsche',
 '@SBhatnagar212',
 '@gp_pulipaka',
 '@MikeTamir',
 'pickover']

In [57]:
topic_nums = tweets['Topic'].value_counts().sort_index().tolist()
topic_nums

[6389,
 7669,
 23089,
 7736,
 8911,
 4846,
 19432,
 17795,
 8143,
 12512,
 18844,
 8541,
 14372,
 18582,
 10668,
 12408,
 14864,
 21736,
 11281,
 7672,
 23252,
 9639,
 8984,
 15191,
 16281,
 15885,
 14946,
 16701,
 15015,
 16418,
 3555,
 9035,
 7723,
 14124,
 6846,
 5907,
 6746,
 15997,
 14132,
 12154,
 8891,
 6133,
 10160,
 13814,
 16168,
 6845,
 18275,
 10217,
 13549,
 9508,
 12071,
 18727,
 10213,
 24393,
 10991,
 7234,
 5434,
 12228,
 16282,
 15893,
 15868,
 8238,
 11267,
 29624,
 12833,
 4652,
 21193,
 16856,
 6938,
 15235,
 9136,
 10692,
 14661,
 5524,
 12284,
 24989,
 7034,
 4293,
 10968,
 4519,
 8260,
 10699,
 3551,
 11514,
 10722,
 11094,
 7882,
 9611,
 11630,
 9010,
 7517,
 9320,
 13828,
 4439,
 19423,
 7721,
 15122,
 9478,
 7252,
 14079]

In [79]:
key_opinion_leaders = []

examples = 10

for n in [97]:
    x = topic_user[topic_user['Topic'] == n].nlargest(examples, ['x'])[['x', 'user']]
    
    a = x['x'].tolist()
    t = x['user'].tolist()

    for i in range(examples):
        key_opinion_leaders.append(
            {
                'topic': n, 
                'user': t[i],
                'tweets_count': a[i], 
                'topic_share': a[i] / topic_nums[n], 
                'user_share': a[i] / users_sum[users_sum['user'] == t[i]]['x'].values[0]
             }
            )
        
pd.DataFrame(key_opinion_leaders).to_csv(repo_path / 'data/to_paper/key_opinion_leaders.csv')


In [77]:
topic_user[topic_user['user'] == '@gp_pulipaka'].sort_values(by='share', ascending=False)

,Topic,user,x,share,user_sum
23524,97,@gp_pulipaka,7910,0.579063,13660
14969,61,@gp_pulipaka,3911,0.286310,13660
14076,57,@gp_pulipaka,332,0.024305,13660
2081,8,@gp_pulipaka,233,0.017057,13660
9878,39,@gp_pulipaka,123,0.009004,13660
...,...,...,...,...,...
15763,65,@gp_pulipaka,1,0.000073,13660
17841,73,@gp_pulipaka,1,0.000073,13660
22840,93,@gp_pulipaka,1,0.000073,13660
21277,87,@gp_pulipaka,1,0.000073,13660


In [78]:
topic_nums[97]

9478

In [83]:
a[0]/topic_nums[97]

0.8345642540620384

In [13]:
sentiment = pd.read_csv(repo_path/'data/sentiments.csv')
sentiment

,sentiment,sentiment_name
0,0,Neutral
1,0,Neutral
2,-1,Negative
3,0,Neutral
4,0,Neutral
...,...,...
1458013,0,Neutral
1458014,0,Neutral
1458015,0,Neutral
1458016,0,Neutral


In [14]:
tweets = tweets.reset_index()
tweets['id'] = tweets.index
sentiment['id'] = sentiment.index

tweets = tweets.join(sentiment.set_index('id'), on='id')
tweets

,level_0,index,user,timestamp,url,txt,original_text,Topic,id,sadness,...,anticipation,joy,fear,optimism,surprise,trust,love,x,sentiment,sentiment_name
0,0,0,pmddomingos,2023-02-16 06:30:30+00:00,https://twitter.com/pmddomingos/status/1626106...,Zero.,@prageru Zero.,59,0,0.443130,...,0.096531,0.035201,0.029071,0.027238,0.015352,0.004598,0.002790,1,0,Neutral
1,1,1,pmddomingos,2023-02-16 06:24:13+00:00,https://twitter.com/pmddomingos/status/1626105...,Any that only needs one world.,@elan_learns Any that only needs one world.,28,1,0.128643,...,0.189322,0.609163,0.014694,0.955574,0.009901,0.224002,0.071346,1,0,Neutral
2,2,2,pmddomingos,2023-02-15 23:26:25+00:00,https://twitter.com/pmddomingos/status/1626000...,The many-worlds interpretation is the most egr...,The many-worlds interpretation is the most egr...,83,2,0.210733,...,0.069136,0.013422,0.109616,0.012648,0.071797,0.007475,0.004135,1,-1,Negative
3,3,3,pmddomingos,2023-02-15 20:54:51+00:00,https://twitter.com/pmddomingos/status/1625961...,The fight over what AIs say and do has just st...,The fight over what AIs say and do has just st...,27,3,0.073350,...,0.190564,0.022842,0.062051,0.398840,0.009873,0.057184,0.005672,1,0,Neutral
4,4,4,pmddomingos,2023-02-15 20:11:47+00:00,https://twitter.com/pmddomingos/status/1625951...,A must-read: 's thoughts on a life in probabil...,A must-read: @heckerma1's thoughts on a life i...,27,4,0.013998,...,0.844487,0.719005,0.070644,0.548011,0.206510,0.217610,0.035281,1,0,Neutral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1458013,1458013,1458013,tamarkasriel,2022-11-04 08:19:36+00:00,https://twitter.com/TamarKasriel/status/158844...,Carrefour Romania discovered 'omnichannel' off...,Carrefour Romania discovered 'omnichannel' off...,5,1458013,0.016256,...,0.743270,0.593059,0.012733,0.556883,0.140772,0.153712,0.017389,1,0,Neutral
1458014,1458014,1458014,tamarkasriel,2022-05-11 12:42:45+00:00,https://twitter.com/TamarKasriel/status/152436...,"Online retail has a ceiling, limits set by a s...","Online retail has a ceiling, limits set by a s...",74,1458014,0.030171,...,0.154668,0.970569,0.007091,0.878854,0.030931,0.073117,0.066877,1,0,Neutral
1458015,1458015,1458015,tamarkasriel,2021-12-10 12:20:10+00:00,https://twitter.com/TamarKasriel/status/146928...,"2021 we were looking for affirmation, soulmate...","2021 we were looking for affirmation, soulmate...",57,1458015,0.041751,...,0.297747,0.829711,0.038546,0.980617,0.019914,0.371656,0.089728,1,0,Neutral
1458016,1458016,1458016,tamarkasriel,2021-08-05 21:34:04+00:00,https://twitter.com/TamarKasriel/status/142339...,"""your personal inbox gradually has become less...","""your personal inbox gradually has become less...",86,1458016,0.108391,...,0.609593,0.049411,0.037524,0.325822,0.049621,0.043266,0.003716,1,0,Neutral


In [23]:
sentiment_stats = tweets.groupby('Topic').mean().drop(columns=['id', 'index', 'level_0'] + emotions.columns.tolist())

/tmp/ipykernel_661312/2612067395.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  sentiment_stats = tweets.groupby('Topic').mean().drop(columns=['id', 'index', 'level_0'] + emotions.columns.tolist())


In [24]:
mapp = tweets.groupby('Topic', as_index=False).value_counts(subset=['sentiment_name'])

for i in range(100):
    sentiment_stats.loc[i, 'Negative'] = mapp[(mapp['Topic'] == i) & (mapp['sentiment_name'] == 'Negative')]['count'].values[0]
    sentiment_stats.loc[i, 'Neutral'] = mapp[(mapp['Topic'] == i) & (mapp['sentiment_name'] == 'Neutral')]['count'].values[0]
    sentiment_stats.loc[i, 'Positive'] = mapp[(mapp['Topic'] == i) & (mapp['sentiment_name'] == 'Positive')]['count'].values[0]
    


In [25]:
sentiment_stats = sentiment_stats.rename(columns={'sentiment':'mean_sentiment'})

In [26]:
sentiment_stats['emotional_ratio'] = sentiment_stats.apply(lambda x: (x['Positive'] + x['Negative'])/(x['Positive'] + x['Negative'] + x['Neutral']), axis=1)

In [27]:
sentiment_stats['pos_neg_ratio'] = sentiment_stats.apply(lambda x: (x['Positive'])/(x['Positive'] + x['Negative']), axis=1)

In [29]:
sentiment_stats['neg_pos_ratio'] = sentiment_stats.apply(lambda x: (x['Negative'])/(x['Positive'] + x['Negative']), axis=1)

In [31]:
sentiment_stats.to_csv(repo_path/'data/to_paper/sentiment_stats.csv')


In [30]:
sentiment_stats

,x,mean_sentiment,Negative,Neutral,Positive,emotional_ratio,pos_neg_ratio,neg_pos_ratio
Topic,,,,,,,,
0,1.0,0.028705,2369.0,21587.0,3147.0,0.203520,0.570522,0.429478
1,1.0,0.351233,1398.0,9938.0,8292.0,0.493682,0.855728,0.144272
2,1.0,-0.061428,1204.0,11761.0,384.0,0.118960,0.241814,0.758186
3,1.0,0.317450,371.0,10245.0,5481.0,0.363546,0.936603,0.063397
4,1.0,-0.039097,2927.0,8604.0,2383.0,0.381630,0.448776,0.551224
...,...,...,...,...,...,...,...,...
95,1.0,0.091235,801.0,10550.0,2021.0,0.211038,0.716159,0.283841
96,1.0,0.467777,1616.0,4564.0,8468.0,0.688422,0.839746,0.160254
97,1.0,0.189705,677.0,14514.0,4392.0,0.258847,0.866443,0.133557
